Generate evaluation triplets from studies 


1. parse document 
2. extract claims that are accosiated with refereces via LLM and save in json with referenced paper as name
3. generate topic of claim like (sustanability, business, money, or something like that)
4. repeat step 1-3 for multiple reports from diverse topics that care be releated tho like (genai, sustability, business report)
5. bundle 2-3 claims together from multiple related documents into one claim via LLM
6. generate question for bundled claim 
7. store question, claim and added references in json objects
8. repeat step 6 and 7




In [ ]:

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.schema import Document
import os
import json
from typing import List, Dict, Any



ImportError: cannot import name 'PyPaperBot' from 'PyPaperBot' (e:\Repositories\graphrag-businessqa-evaluation\env\Lib\site-packages\PyPaperBot\__init__.py)

In [ ]:

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client('gen-ai-hub')
llm = ChatOpenAI(proxy_model_name='gpt-4o', proxy_client=proxy_client)

INPUT_DIR = "reports/"
OUTPUT_FILE = "evaluation_references.json"




## Load Documents

In [6]:
documents = []
for file in os.listdir(INPUT_DIR):
    if file.endswith(".md"):
        file_path = os.path.join(INPUT_DIR, file)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            documents.append(Document(page_content=content, metadata={"source": file}))


## Extract claims with references using an LLM chain

In [7]:
import os
import json

from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document


class Response(BaseModel):
    claim: str = Field(description="Claim"),
    references: List[str] = Field(description="List of references")
    dois: List[str] = Field(description="List of DOIs corresponding to each reference in the same order as references, or empty if not found")

class Responses(BaseModel):
    responses: list[Response] = Field(description="List of responses")

parser = PydanticOutputParser(pydantic_object=Responses)

extract_prompt = PromptTemplate(
    input_variables=["text"],
    partial_variables={"format_output": parser.get_format_instructions()},
    template="""You are a knowledgeable assistant specializing in academic research analysis. Your task is to identify and extract **research-based claims** from the provided text. Additionally, you will match each inline citation in the claim to its corresponding entry in the references section and extract the DOI from that entry.

**Definition of a Research-Based Claim:**
- A statement that presents a finding, result, or implication derived from empirical studies.
- Typically includes analysis, comparisons, or conclusions drawn from data.
- Supported by one or more inline citations in the format "(Author et al., YEAR)".

**Exclude the following from extraction:**
- Definitions or historical accounts.
- General statements without empirical support.
- Descriptions of concepts without presenting findings.

**Tasks:**
1. Extract each research-based claim without its inline citations.
2. For each claim, extract the list of inline citations that appear in that claim.
3. For each inline citation (Author et al., YEAR), match it to the corresponding full reference in the references section (if available within the provided text).
4. From the matched full reference, extract the DOI (if present).
5. If no DOI is found for a given citation, return an empty string for that DOI.

**Input:**
{text}


**The output MUST strictly adhere to the following JSON format and include only this JSON without any additional text:**
{format_output}
""", 
)

extract_chain = extract_prompt | llm | parser

claims = []
for doc in documents:
    response = extract_chain.invoke({"text": doc.page_content})
    for res in response.responses:
        claim_dict = res.dict()
        claim_dict["doc_source"] = doc.metadata["source"]
        claims.append(claim_dict)




e:\Repositories\graphrag-businessqa-evaluation\env\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='Claim'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [39]:
print(claims)

[{'claim': 'Endiana et al. affirm that the accounting sector of a company can promote environmental conservation through environmental costs, and at the same time, improve performance when implementing a Corporate Sustainability Management System (CSMS). Mainly, Endiana et al. evidence that allocating appropriate environmental costs through CSMS can effectively improve the company’s financial performance. Besides, it is believed that a proper application of CSMS, with the disclosure of environmental activities and costs (e.g., of land, materials, energy, etc.), can enhance customer loyalty.', 'references': ['Endiana, I., Dicriyani, N. L. G. M., Adiyadnya, M. S. P., & Putra, I. P. M. J. S. (2020). The effect of green accounting on corporate sustainability and financial performance. The Journal of Asian Finance, Economics, and Business, 7(12), 731–738. https://doi.org/10.13106/jafeb.2020.vol7.no12.731'], 'dois': ['https://doi.org/10.13106/jafeb.2020.vol7.no12.731'], 'doc_source': 's10668

## Fetch and filter cited Reports 

In [ ]:
import os
from scidownl import scihub_download

download_dir = "citet_papers/"
os.makedirs(download_dir, exist_ok=True)

downloaded_dois = set()
filtered_claims = []

for claim in claims:
    valid_dois = [doi for doi in claim["dois"] if doi]
    if valid_dois:
        filtered_claims.append(claim)

for claim in filtered_claims:
    for doi in claim["dois"]:
        if doi not in downloaded_dois:
            try:

                scihub_download(doi, paper_type="doi", out=download_dir)
                downloaded_dois.add(doi)
                print(f"Downloaded: {doi}")
            except Exception as e:
                print(f"Failed to download {doi}: {e}")

# update claims to only include those with successfully downloaded PDFs
final_claims = [
    claim for claim in filtered_claims
    if all(doi in downloaded_dois for doi in claim["dois"])
]

print(final_claims)


[INFO] | 2024/12/09 00:36:25 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2024/12/09 00:36:25 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=doi.org/10.13106/jafeb.2020.vol7.no12.731], proxies={}
[INFO] | 2024/12/09 00:36:25 | -> Response: status_code=200, content_length=5842
[WARNING] | 2024/12/09 00:36:25 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/12/09 00:36:25 | Choose scihub url [1]: http://sci-hub.st
[INFO] | 2024/12/09 00:36:25 | <- Request: scihub_url=http://sci-hub.st, source=DoiSource[type=doi, id=doi.org/10.13106/jafeb.2020.vol7.no12.731], proxies={}
[INFO] | 2024/12/09 00:36:25 | -> Response: status_code=200, content_length=5842
[WARNING] | 2024/12/09 00:36:25 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/12/09 00:36:25 | Choose scihub url [2]: http://sci-hub.s

Downloaded: https://doi.org/10.13106/jafeb.2020.vol7.no12.731


[INFO] | 2024/12/09 00:36:27 | -> Response: status_code=200, content_length=7974
[INFO] | 2024/12/09 00:36:27 | * Extracted information: {'url': 'https://sci-hub.ru/downloads/2020-12-23/2e/10.3390@su122410514.pdf', 'title': 'Corporate Sustainability  It’s Mine! Effect of Green Product Psychological Ownership on the Environmental Behavior and Performance of Employees. Sustainability, 12(24), 10514'}


100% [==================================================] 603723/603723


[INFO] | 2024/12/09 00:36:27 | ↓ Successfully download the url to: citet_papers\Corporate Sustainability  It’s Mine! Effect of Green Product Psychological Ownership on the Environmental Behavior and Performance of Employees. Sustainability, 12(24), 10514.pdf


...

Downloaded: https://doi.org/10.5465/amr.2012.0341


[INFO] | 2024/12/09 00:36:34 | -> Response: status_code=200, content_length=7841
[INFO] | 2024/12/09 00:36:34 | * Extracted information: {'url': 'https://dacemirror.sci-hub.ru/journal-article/fdde4e646a22395654d744f8a8b0ea9c/baumgartner2016.pdf', 'title': ''}


100% [==================================================] 2692460/2692460


[INFO] | 2024/12/09 00:36:35 | ↓ Successfully download the url to: citet_papers\.pdf


Downloaded: https://doi.org/10.1016/j.jclepro.2016.04.146
[{'claim': 'Endiana et al. affirm that the accounting sector of a company can promote environmental conservation through environmental costs, and at the same time, improve performance when implementing a Corporate Sustainability Management System (CSMS). Mainly, Endiana et al. evidence that allocating appropriate environmental costs through CSMS can effectively improve the company’s financial performance. Besides, it is believed that a proper application of CSMS, with the disclosure of environmental activities and costs (e.g., of land, materials, energy, etc.), can enhance customer loyalty.', 'references': ['Endiana, I., Dicriyani, N. L. G. M., Adiyadnya, M. S. P., & Putra, I. P. M. J. S. (2020). The effect of green accounting on corporate sustainability and financial performance. The Journal of Asian Finance, Economics, and Business, 7(12), 731–738. https://doi.org/10.13106/jafeb.2020.vol7.no12.731'], 'dois': ['https://doi.org/

## Categorize the claims 

In [38]:
from typing import List
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

class CategoryResponse(BaseModel):
    category: str = Field(description="The category of the given claim")

parser = PydanticOutputParser(pydantic_object=CategoryResponse)

extract_prompt = PromptTemplate(
    input_variables=["claim", "categories"],
    partial_variables={"format_output": parser.get_format_instructions()},
    template="""You have a list of existing categories:
{categories}

Please classify the following claim into one of these categories if it fits well. 
If it does not fit into any of the listed categories, create a new category name.
Here are some examples for Categories you can use, or create new ones if you need to:
"Business", "Sustainability", "Technology", "Economics", "Innovation", etc.

Claim: {claim}

Return only the final chosen or newly created category as a JSON object following the format instructions.

{format_output}
"""
)

extract_chain = extract_prompt | llm | parser

categories = []
final_claims = []
for item in claims:
    categories_str = ", ".join(categories) if categories else "No categories yet."
    response = extract_chain.invoke({"claim": item["claim"], "categories": categories_str})
    chosen_category = response.category.strip()
    if chosen_category not in categories:
        categories.append(chosen_category)

    item['category'] = chosen_category
    final_claims.append(item)

print("Final Claims with Categories:", final_claims)
print("All Discovered Categories:", categories)
print("Amount of claims:", len(claims))

Final Claims with Categories: [{'claim': 'Endiana et al. affirm that the accounting sector of a company can promote environmental conservation through environmental costs, and at the same time, improve performance when implementing a Corporate Sustainability Management System (CSMS). Mainly, Endiana et al. evidence that allocating appropriate environmental costs through CSMS can effectively improve the company’s financial performance. Besides, it is believed that a proper application of CSMS, with the disclosure of environmental activities and costs (e.g., of land, materials, energy, etc.), can enhance customer loyalty.', 'references': ['Endiana, I., Dicriyani, N. L. G. M., Adiyadnya, M. S. P., & Putra, I. P. M. J. S. (2020). The effect of green accounting on corporate sustainability and financial performance. The Journal of Asian Finance, Economics, and Business, 7(12), 731–738. https://doi.org/10.13106/jafeb.2020.vol7.no12.731'], 'dois': ['https://doi.org/10.13106/jafeb.2020.vol7.no1

## Bundle and Merge Claims

In [ ]:
from collections import defaultdict
from langchain.prompts import PromptTemplate

for idx, claim in enumerate(final_claims):
    claim["id"] = idx

claims_by_category = defaultdict(list)
for c in final_claims:
    claims_by_category[c["category"]].append(c)

all_claims = [c for cat_list in claims_by_category.values() for c in cat_list]

merge_prompt = PromptTemplate(
    input_variables=["claims_text"],
    template="""You are a writing assistant.
Merge the following claims into a single cohesive claim that captures all their key points but dont summarize them.:

Claims:
{claims_text}

Return one paragraph unifying these claims.
""")


group_size = 3
merged_claims = []

def get_other_category_claim(current_category, used_claims):
    for c in all_claims:
        if c["category"] != current_category and c["id"] not in used_claims:
            return c
    return None

used_claims = set()  

for cat, cat_claims in claims_by_category.items():
    for i in range(0, len(cat_claims), group_size):
        group = cat_claims[i:i+group_size]
        if len(set(g["category"] for g in group)) == 1:
            if len(group) < group_size:
                other_claim = get_other_category_claim(cat, used_claims)
                if other_claim is not None:
                    group.append(other_claim)
                    used_claims.add(other_claim["id"])
            else:
                other_claim = get_other_category_claim(cat, used_claims)
                if other_claim is not None:
                    replaced = group.pop()  
                    group.append(other_claim)
                    used_claims.add(other_claim["id"])

        for g in group:
            used_claims.add(g["id"])

        claims_text = "\n".join([f"- {g['claim']}" for g in group])
        merged_claim = (merge_prompt | llm).invoke({"claims_text": claims_text})

        all_refs = []
        for g in group:
            all_refs.extend(g.get("references", []))

        merged_claims.append({
            "original_claims": group,
            "merged_claim": merged_claim.content.strip(),
            "category": cat,
            "merged_references": list(set(all_refs))
        })

print("Merged Claims:")
for mc in merged_claims:
    print(mc)


Merged Claims:
{'original_claims': [{'claim': 'Endiana et al. affirm that the accounting sector of a company can promote environmental conservation through environmental costs, and at the same time, improve performance when implementing a Corporate Sustainability Management System (CSMS). Mainly, Endiana et al. evidence that allocating appropriate environmental costs through CSMS can effectively improve the company’s financial performance. Besides, it is believed that a proper application of CSMS, with the disclosure of environmental activities and costs (e.g., of land, materials, energy, etc.), can enhance customer loyalty.', 'references': ['Endiana, I., Dicriyani, N. L. G. M., Adiyadnya, M. S. P., & Putra, I. P. M. J. S. (2020). The effect of green accounting on corporate sustainability and financial performance. The Journal of Asian Finance, Economics, and Business, 7(12), 731–738. https://doi.org/10.13106/jafeb.2020.vol7.no12.731'], 'dois': ['https://doi.org/10.13106/jafeb.2020.vol

In [ ]:

question_promt = PromptTemplate(
    input_variables=["merged_claim"],
    template=
    """You have the following merged claim:
    "{merged_claim}"
    
    Generate a single, question that, if answered, would be naturally resolved by this claim.
    This question should:
    - Integrate insights from multiple logically connected business domains (e.g., finance, sustainability, technology, operations).
    - Require synthesizing and validating information potentially from multiple sections or sources, reflecting the complexity of cross-document or intra-document retrieval.
    - Demand abstract reasoning and strategic-level interpretation, going beyond simple fact retrieval.
    
    The resulting question should be global in scope, and should not be answerable without the information provided in the claim.
    """
)

final_dataset = []

for mc in merged_claims:
    question_response = (question_promt | llm).invoke({"merged_claim": mc["merged_claim"]})
    question = question_response.content.strip()
    
    entry = {
        "question": question,
        "ground_truth": mc["merged_claim"],
        "references": mc["merged_references"]
    }
    
    final_dataset.append(entry)


with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump({"responses": final_dataset}, f, indent=4, ensure_ascii=False)


for entry in final_dataset:
    print(json.dumps(entry, indent=4, ensure_ascii=False))

{
    "question": "How can a company strategically integrate a Corporate Sustainability Management System (CSMS) with a Green Shared Vision and a comprehensive evaluation framework to simultaneously enhance environmental conservation, improve financial performance, and foster stronger stakeholder relationships, while addressing specific challenges such as green confusion, social sustainability, and the impact of external factors like climate vulnerability and pandemics?",
    "ground_truth": "Endiana et al. assert that the accounting sector of a company can foster environmental conservation and enhance performance through the implementation of a Corporate Sustainability Management System (CSMS), by appropriately allocating environmental costs, thereby improving financial outcomes and customer loyalty. From the employees' perspective, Chang et al. emphasize the importance of establishing a solid Green Shared Vision to mitigate green confusion and boost economic performance, highlighting